In [2]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings("ignore")

# If the scraper doesn't fully run, do it by year and run the following code block

In [3]:
import_Data = []
count=0
for file in os.listdir('./Data'):

    if file.startswith('NFLData'):

        dp = pd.read_csv(f'./Data/{file}')
        dp['season'] = count + 1

        import_Data.append(dp)
        count += 1

df = pd.concat(import_Data, ignore_index=True)

df.to_csv('./Data/Combined_NFLData.csv', index=False)

In [4]:
# Otherwise

df = pd.read_csv("./Data/Combined_NFLData.csv")
df

,game_url,away_team_name,away_team_initials,away_record_status,away_Q1,away_Q2,away_Q3,away_Q4,away_final_score,home_team_name,...,home_Q2,home_Q3,home_Q4,home_final_score,stadium_name,stadium_attendance,stadium_capacity,game_location,game_date,season
0,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Kansas City Chiefs,KC,"1-0, 1-0 away",7.0,7.0,7.0,21.0,42.0,New England Patriots,...,10.0,10.0,0.0,27.0,Gillette Stadium,"65,878","65,878","Foxboro, MA","8:30 PM, September 7, 2017",1
1,https://www.espn.com/nfl/playbyplay/_/gameId/4...,New York Jets,NYJ,"0-1, 0-1 away",0.0,6.0,6.0,0.0,12.0,Buffalo Bills,...,7.0,7.0,7.0,21.0,Highmark Stadium,"68,751","71,621","Orchard Park, NY","1:00 PM, September 10, 2017",1
2,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Atlanta Falcons,ATL,"1-0, 1-0 away",3.0,7.0,3.0,10.0,23.0,Chicago Bears,...,10.0,0.0,7.0,17.0,Soldier Field,"61,857","61,500","Chicago, IL","1:00 PM, September 10, 2017",1
3,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Baltimore Ravens,BAL,"1-0, 1-0 away",3.0,14.0,3.0,0.0,20.0,Cincinnati Bengals,...,0.0,0.0,0.0,0.0,Paycor Stadium,"55,254","65,515","Cincinnati, OH","1:00 PM, September 10, 2017",1
4,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Pittsburgh Steelers,PIT,"1-0, 1-0 away",7.0,7.0,7.0,0.0,21.0,Cleveland Browns,...,0.0,3.0,8.0,18.0,Cleveland Browns Stadium,"67,431","67,431","Cleveland, OH","1:00 PM, September 10, 2017",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Dallas Cowboys,DAL,"12-5, 4-4 away",0,6,3,3,12.0,San Francisco 49ers,...,6,0,10,19.0,Levi's Stadium,"71,888","68,500","Santa Clara, CA","6:30 PM, January 22, 2023",6
1643,https://www.espn.com/nfl/playbyplay/_/gameId/4...,San Francisco 49ers,SF,"13-4, 5-3 away",0,7,0,0,7.0,Philadelphia Eagles,...,14,7,3,31.0,Lincoln Financial Field,"69,879","69,879","Philadelphia, PA","3:00 PM, January 29, 2023",6
1644,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Cincinnati Bengals,CIN,"12-4, 6-3 away",0,6,7,7,20.0,Kansas City Chiefs,...,10,7,3,23.0,GEHA Field at Arrowhead Stadium,"73,426","72,936","Kansas City, MO","6:30 PM, January 29, 2023",6
1645,https://www.espn.com/nfl/playbyplay/_/gameId/4...,AFC,AFC,NaN,,,,,33.0,NFC,...,,,,35.0,Allegiant Stadium,NaN,"65,000","Las Vegas, NV","3:00 PM, February 5, 2023",6


In [5]:
# Change game date to date time

df['game_date'] = df['game_date'].str.replace('TBD, ', '')
game_date = pd.to_datetime(df['game_date'])

df['game_date'] = pd.to_datetime(game_date, format='%d-%m-%Y')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647 entries, 0 to 1646
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   game_url            1647 non-null   object        
 1   away_team_name      1647 non-null   object        
 2   away_team_initials  1644 non-null   object        
 3   away_record_status  1641 non-null   object        
 4   away_Q1             1644 non-null   object        
 5   away_Q2             1644 non-null   object        
 6   away_Q3             1644 non-null   object        
 7   away_Q4             1644 non-null   object        
 8   away_final_score    1644 non-null   float64       
 9   home_team_name      1647 non-null   object        
 10  home_team_initials  1644 non-null   object        
 11  home_record_status  1641 non-null   object        
 12  home_Q1             1644 non-null   object        
 13  home_Q2             1644 non-null   object      

# Cleaning the data

game_date column to date_time


- remove nfc, afc rows as well as away_team_initials == '0'

In [6]:
# There were some error's liked canceled games and afc/nfc games. These don't really matter much, so we might as well remove them.
print(df.shape)
df =df[(df['away_team_initials'] != '0') & (~df['away_team_initials'].isin(['AFC', 'NFC']))]
print(df.shape)

(1647, 23)
(1642, 23)


In [7]:
replacement = {1: 2017, 2:2018, 3:2019, 4:2020, 5:2021, 6:2022}

df.loc[:, 'season'] = df['season'].replace(replacement)

In [8]:
df.isna().sum()

game_url                0
away_team_name          0
away_team_initials      3
away_record_status      1
away_Q1                 3
away_Q2                 3
away_Q3                 3
away_Q4                 3
away_final_score        3
home_team_name          0
home_team_initials      3
home_record_status      1
home_Q1                 3
home_Q2                 3
home_Q3                 3
home_Q4                 3
home_final_score        3
stadium_name            0
stadium_attendance    156
stadium_capacity        0
game_location           0
game_date               0
season                  0
dtype: int64

In [9]:
df['stadium_attendance'] = df['stadium_attendance'].fillna('0,')

In [10]:
df = df.dropna()
df.isna().sum()

game_url              0
away_team_name        0
away_team_initials    0
away_record_status    0
away_Q1               0
away_Q2               0
away_Q3               0
away_Q4               0
away_final_score      0
home_team_name        0
home_team_initials    0
home_record_status    0
home_Q1               0
home_Q2               0
home_Q3               0
home_Q4               0
home_final_score      0
stadium_name          0
stadium_attendance    0
stadium_capacity      0
game_location         0
game_date             0
season                0
dtype: int64

In [11]:
df = df.reset_index(drop=True)

q1 - q4 home and away, final home and away, stadium attendance, capacity

In [12]:
df['stadium_attendance'] = df['stadium_attendance'].str.replace(',', '').astype(int)
df['stadium_capacity'] = df['stadium_capacity'].str.replace(',', '').astype(int)

In [13]:
# Need this to calculate the home and away team win percentage

home_team_win_perc = []
away_team_win_perc = []


for s in df['away_record_status']:
    scores = s.split(', ')[0].split('-')
    
    if len(scores) == 2:
        data = [int(scores[0]), int(scores[1]), 0]
        s = sum(data)
        away_team_win_perc.append(float('{:.2f}'.format((data[0]/s)*100)))

    elif len(scores) == 3:
        data = [int(scores[0]), int(scores[1]), int(scores[2])]
        s = sum(data)
        away_team_win_perc.append(float('{:.2f}'.format((data[0]/s)*100)))

for s in df['home_record_status']:
    scores = s.split(', ')[0].split('-')
    
    if len(scores) == 2:
        data = [int(scores[0]), int(scores[1]), 0]
        s = sum(data)
        home_team_win_perc.append(float('{:.2f}'.format((data[0]/s)*100)))

    elif len(scores) == 3:
        data = [int(scores[0]), int(scores[1]), int(scores[2])]
        s = sum(data)
        home_team_win_perc.append(float('{:.2f}'.format((data[0]/s)*100)))
    

df['away_team_win_perc'] = away_team_win_perc
df['home_team_win_perc'] = home_team_win_perc

df = df.drop(['away_record_status', 'home_record_status'], axis=1)

In [14]:
l = ['home_Q1', 'home_Q2', 'home_Q3', 'home_Q4', 'home_final_score', 'away_Q1', 'away_Q2', 'away_Q3', 'away_Q4', 'away_final_score']

for x in l:
    df[x] = df[x].apply(lambda a: int(float(a)))

In [15]:
df['season'] = df['season'].astype('category')

All columns are up to date, now we want to do some....


# Feature engineering

In [16]:
q1=[]
q2=[]
q3=[]
q4=[]
f=[]
home_avg=[] # Average points per game for the home team
away_avg=[] # Average points per game for the away team
for x, game in df.iterrows():
    home_score = [game['home_Q1'], game['home_Q2'], game['home_Q3'], game['home_Q4']]
    away_score = [game['away_Q1'], game['away_Q2'], game['away_Q3'], game['away_Q4']]
    w1,w2,w3,w4 = [abs(home_score[i] - away_score[i]) for i in range(4)]

    final_diff = abs(game['home_final_score'] - game['away_final_score'])

    home_avg.append(np.mean(home_score))
    away_avg.append(np.mean(away_score))


    q1.append(w1)
    q2.append(w2)
    q3.append(w3)
    q4.append(w4)
    f.append(final_diff)


df['Q1_diff'] = q1
df['Q2_diff'] = q2
df['Q3_diff'] = q3
df['Q4_diff'] = q4
df['final_score_diff'] = f
df['home_avg'] = home_avg
df['away_avg'] = away_avg

In [17]:
winner = []
for i, row in df[['home_final_score', 'away_final_score']].iterrows():
    home = row['home_final_score']
    away = row['away_final_score']

    if home > away:
        winner.append(1) # Home
    else:
        winner.append(0) # Away

df['winner'] = winner

In [18]:
from geopy.geocoders import Nominatim

valid_locations = {
'San Francisco 49ers': 'Santa Clara, CA',
'Tennessee Titans': 'Nashville, TN',
'Seattle Seahawks': 'Seattle, WA',
'Jacksonville Jaguars': 'Jacksonville, FL',
'New York Giants': 'East Rutherford, NJ',
'Cincinnati Bengals': 'Cincinnati, OH',
'Los Angeles Rams': 'Inglewood, CA',
'Pittsburgh Steelers': 'Pittsburgh, PA',
'Dallas Cowboys': 'Arlington, TX',
'Indianapolis Colts': 'Indianapolis, IN',
'Houston Texans': 'Houston, TX',
'Buffalo Bills': 'Orchard Park, NY',
'New England Patriots': 'Foxborough, MA',
'Cleveland Browns': 'Cleveland, OH',
'Miami Dolphins': 'Miami Gardens, FL',
'Green Bay Packers': 'Green Bay, WI',
'Minnesota Vikings': 'Minneapolis, MN',
'Philadelphia Eagles': 'Philadelphia, PA',
'New Orleans Saints': 'New Orleans, LA',
'Arizona Cardinals': 'Glendale, AZ',
'Los Angeles Chargers': 'Inglewood, CA',
'Detroit Lions': 'Detroit, MI',
'Kansas City Chiefs': 'Kansas City, MO',
'Denver Broncos': 'Denver, CO',
'New York Jets': 'East Rutherford, NJ',
'Baltimore Ravens': 'Baltimore, MD',
'Atlanta Falcons': 'Atlanta, GA',
'Carolina Panthers': 'Charlotte, NC',
'Tampa Bay Buccaneers': 'Tampa, FL',
'Chicago Bears': 'Chicago, IL',
'Oakland Raiders': 'Oakland, CA',
'Las Vegas Raiders': 'Paradise, NV',
'Washington Redskins': 'Landover, MD',
'Washington': 'Landover, MD',
'Washington Commanders': 'Landover, MD'
}

geolocator = Nominatim(user_agent='my_app')


teams = set(df['away_team_name'])
team_coords = {}

for team in teams:
    location = geolocator.geocode(valid_locations[team])

    team_coords[team] = (location.latitude, location.longitude)

team_coords

{'Houston Texans': (29.7589382, -95.3676974),
 'Pittsburgh Steelers': (40.4416941, -79.9900861),
 'Carolina Panthers': (35.2272086, -80.8430827),
 'Tampa Bay Buccaneers': (27.9477595, -82.458444),
 'Green Bay Packers': (44.5126379, -88.0125794),
 'Baltimore Ravens': (39.2908816, -76.610759),
 'Chicago Bears': (41.8755616, -87.6244212),
 'Washington Redskins': (38.92531295, -76.88154330442846),
 'Seattle Seahawks': (47.6038321, -122.330062),
 'San Francisco 49ers': (37.2333253, -121.6846349),
 'Arizona Cardinals': (33.5386858, -112.185994),
 'Las Vegas Raiders': (36.1150858, -115.173414),
 'Denver Broncos': (39.7392364, -104.984862),
 'Minnesota Vikings': (44.9772995, -93.2654692),
 'Washington Commanders': (38.92531295, -76.88154330442846),
 'New Orleans Saints': (29.9759983, -90.0782127),
 'Oakland Raiders': (37.8044557, -122.271356),
 'Los Angeles Rams': (33.9562003, -118.353132),
 'Cincinnati Bengals': (39.1014537, -84.5124602),
 'Dallas Cowboys': (32.7355816, -97.1071186),
 'Indian

In [19]:
cities = set(df['game_location'])
city_coords = {}

for city in cities:
    loc = geolocator.geocode(city.strip())

    city_coords[city] = (loc.latitude, loc.longitude)

city_coords

{'Cleveland, OH ': (41.4996574, -81.6936772),
 'Orchard Park, NY ': (42.7670376, -78.7435517),
 'Charlotte, NC ': (35.2272086, -80.8430827),
 'Los Angeles, CA ': (34.0536909, -118.242766),
 'Atlanta, GA ': (33.7489924, -84.3902644),
 'Mexico City ': (19.4326296, -99.1331785),
 'Pittsburgh, PA ': (40.4416941, -79.9900861),
 'Glendale, AZ ': (33.5386858, -112.185994),
 'Arlington, TX ': (32.7355816, -97.1071186),
 'Houston, TX ': (29.7589382, -95.3676974),
 'Landover, MD ': (38.92531295, -76.88154330442846),
 'Chicago, IL ': (41.8755616, -87.6244212),
 'New Orleans, LA ': (29.9759983, -90.0782127),
 'East Rutherford, NJ ': (40.833989, -74.0970865),
 'Seattle, WA ': (47.6038321, -122.330062),
 'Oakland, CA ': (37.8044557, -122.271356),
 'Green Bay, WI ': (44.5126379, -88.0125794),
 'Foxboro, MA ': (46.4985528, -92.2879727),
 'Miami Gardens, FL ': (25.9420377, -80.2456045),
 'Munich ': (48.1371079, 11.5753822),
 'Minneapolis, MN ': (44.9772995, -93.2654692),
 'Las Vegas, NV ': (36.1672559,

In [20]:
df['lat_away'] = df['away_team_name'].map(lambda x: team_coords[x][0])
df['long_away'] = df['away_team_name'].map(lambda x: team_coords[x][1])

df['lat_home'] = df['home_team_name'].map(lambda x: team_coords[x][0])
df['long_home'] = df['home_team_name'].map(lambda x: team_coords[x][1])

df['lat_game'] = df['game_location'].map(lambda x: city_coords[x][0])
df['long_game'] = df['game_location'].map(lambda x: city_coords[x][1])

In [21]:
from geopy.distance import distance

def calc_distance_away(row):
    team = (row['lat_away'], row['long_away'])
    game = (row['lat_game'], row['long_game'])
    return round(distance(team, game).miles, 2)

def calc_distance_home(row):
    team = (row['lat_home'], row['long_home'])
    game = (row['lat_game'], row['long_game'])
    return round(distance(team, game).miles, 2)


df['away_distance_to_game'] = df.apply(calc_distance_away, axis=1)
df['home_distance_to_game'] = df.apply(calc_distance_home, axis=1)

In [22]:
cities=[]
state_code=[]
for loc in df.game_location:
    temp = loc.split(', ')
    if len(temp) == 1:
        if temp[0].strip() == 'London':
            cities.append(temp[0].strip())
            state_code.append('FR')
        elif temp[0].strip() == 'Mexico City':
            cities.append(temp[0].strip())
            state_code.append('MX')
        elif temp[0].strip() == 'Munich':
            cities.append(temp[0].strip())
            state_code.append('DE')
    else:
        cities.append(temp[0])
        state_code.append(temp[1])

df['city'] = cities
df['state_code'] = state_code

df = df.drop('game_location', axis=1)

In [23]:
df = df.fillna(0)
df

,game_url,away_team_name,away_team_initials,away_Q1,away_Q2,away_Q3,away_Q4,away_final_score,home_team_name,home_team_initials,...,lat_away,long_away,lat_home,long_home,lat_game,long_game,away_distance_to_game,home_distance_to_game,city,state_code
0,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Kansas City Chiefs,KC,7,7,7,21,42,New England Patriots,NE,...,39.100105,-94.578142,42.065377,-71.247831,46.498553,-92.287973,523.73,1083.85,Foxboro,MA
1,https://www.espn.com/nfl/playbyplay/_/gameId/4...,New York Jets,NYJ,0,6,6,0,12,Buffalo Bills,BUF,...,40.833989,-74.097087,42.767038,-78.743552,42.767038,-78.743552,274.48,0.00,Orchard Park,NY
2,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Atlanta Falcons,ATL,3,7,3,10,23,Chicago Bears,CHI,...,33.748992,-84.390264,41.875562,-87.624421,41.875562,-87.624421,587.62,0.00,Chicago,IL
3,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Baltimore Ravens,BAL,3,14,3,0,20,Cincinnati Bengals,CIN,...,39.290882,-76.610759,39.101454,-84.512460,39.101454,-84.512460,424.22,0.00,Cincinnati,OH
4,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Pittsburgh Steelers,PIT,7,7,7,0,21,Cleveland Browns,CLE,...,40.441694,-79.990086,41.499657,-81.693677,41.499657,-81.693677,115.19,0.00,Cleveland,OH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Cincinnati Bengals,CIN,14,3,7,3,27,Buffalo Bills,BUF,...,39.101454,-84.512460,42.767038,-78.743552,42.767038,-78.743552,393.68,0.00,Orchard Park,NY
1635,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Dallas Cowboys,DAL,0,6,3,3,12,San Francisco 49ers,SF,...,32.735582,-97.107119,37.233325,-121.684635,37.233325,-121.684635,1424.11,0.00,Santa Clara,CA
1636,https://www.espn.com/nfl/playbyplay/_/gameId/4...,San Francisco 49ers,SF,0,7,0,0,7,Philadelphia Eagles,PHI,...,37.233325,-121.684635,39.952724,-75.163526,39.952724,-75.163526,2496.79,0.00,Philadelphia,PA
1637,https://www.espn.com/nfl/playbyplay/_/gameId/4...,Cincinnati Bengals,CIN,0,6,7,7,20,Kansas City Chiefs,KC,...,39.101454,-84.512460,39.100105,-94.578142,39.100105,-94.578142,540.76,0.00,Kansas City,MO


In [24]:
df.to_csv('./Data/Final Compose_data.csv', index=False)